In [418]:
import csv

In [419]:
export_file = '/Users/jdamerow/UpConsulting/files/latex/full-test.csv' #'/Users/jdamerow/UpConsulting/files/latex/test_file.csv'
result_file_path = '/Users/jdamerow/UpConsulting/files/latex/result.txt'
reviews_file_path = '/Users/jdamerow/UpConsulting/files/latex/reviews.txt'

In [420]:

author_template = "\\textsc{%s}, %s"
isbn_template = "\\textsc{isbn}: %s."

In [421]:
AUTHOR_SEPARATER = "; "
CHAPTER_SEPARATER = "; "

In [422]:
def get_last_first(name):
    if ',' in name:
        last = name[:name.index(',')]
        first = name[name.index(',')+1:]
    else:
        last = name[name.rfind(" "):].strip()
        first = name[:name.rfind(" ")].strip()
    return last, first

In [423]:
class Author:
    def __init__(self):
        self.name = ''
        self.order = 0
        
    def get_last_name(self):
        return get_last_first(self.name)[0]
        
    def get_first_name(self):
        return get_last_first(self.name)[1]
        
    def get_name_for_sort(self):
        return self.get_last_name().lower() + ", " + self.get_first_name().lower()
    
    def get_order():
        return self.order

In [424]:
def build_persons_latex_list(persons):
    persons_latex = []
    if persons:
        for p in persons:
            last, first = get_last_first(p.name)
            if last and first:
                persons_latex.append(author_template%(last, first))
    return persons_latex
  

In [425]:
class Document:
    def __init__(self):
        self.category_number = ''
        self.print_date = ''
        self.index = 0
        self.isiscb_id = ''
        self.title = ''
        self.year = 0

In [426]:
latex_template_book = """
 
 
\\noindent\\begin{footnotesize}\\textbf{%s}\\hspace{0.5em} %s. \\textit{%s}%s. %s, %s. %s \\textsc{\\href{https://data.isiscb.org/isis/citation/%s}{%s}}.
%s

\\vspace{2ex} \\end{footnotesize}
\\paragraph{}

"""

latex_template_book_chapters = """
\\begin{isisdescription} Includes:  %s, 
``\\selectlanguage{frenchb}%s\\selectlanguage{english}''%s%s.\\end{isisdescription}
"""

chapter_ref=" [ref.~%s]"
editors_template = ", %s (Eds.)"
pages_template = " %s"

class Book(Document):
    def __init__(self):
        Document.__init__(self)
        self.authors = []
        self.editors = []
        self.title = ''
        self.year = 0
        self.publisher = ''
        self.isbn = ''
        self.chapters = []
        self.chapter_ids = []
        
        
    def get_sort_by_value(self):
        return "; ".join([a.get_name_for_sort() for a in (self.authors if self.authors else self.editors)])
        
    def get_editor_string(self):
        editors_latex = []
        if self.editors:
            for e in self.editors:
                last, first = get_last_first(e.name)
                if last and first:
                    editors_latex.append(author_template%(last, first))
        eds_suffix = ""
        if editors_latex:
            eds_suffix = " (Eds.)"
            if len(editors_latex) == 1:
                eds_suffix = " (Ed.)"
                
        return AUTHOR_SEPARATER.join(editors_latex) + eds_suffix
        
    def render(self, counter):
        authors_latex = build_persons_latex_list(self.authors)
        editors_latex = build_persons_latex_list(self.editors)
        
        author_editor_string = ""
        editors_string = ""
        if editors_latex:
            eds_suffix = " (Eds.)"
            if len(editors_latex) == 1:
                eds_suffix = " (Ed.)"
                    
        if authors_latex:
            author_editor_string = AUTHOR_SEPARATER.join(authors_latex)
            if editors_latex:
                editors_string = ", " + AUTHOR_SEPARATER.join(editors_latex) + eds_suffix
        elif editors_latex:
            author_editor_string = AUTHOR_SEPARATER.join(editors_latex) + eds_suffix

        isbn = ''
        if self.isbn:
            isbn = isbn_template%(self.isbn)
            
        # render chapters
        chapter_latex = []
        if self.chapters:
            for chap in self.chapters:
                chapter_authors = build_persons_latex_list(chap.authors)
                
                chap_pages = ''
                if chap.pages:
                    chap_pages = pages_template%(chap.pages)
                    
                chap_ref = ""
                if chap.index:
                    chap_ref = chapter_ref%(chap.index)

                chapter_latex.append(latex_template_book_chapters%(AUTHOR_SEPARATER.join(chapter_authors), chap.title, chap_pages, chap_ref))
            
        return latex_template_book%(counter, author_editor_string, self.title, editors_string,  
                                    self.publisher, self.year, isbn, self.isiscb_id, self.isiscb_id, CHAPTER_SEPARATER.join(chapter_latex))
                                    


In [427]:
latex_template_article = """
 
 
\\noindent\\begin{footnotesize}\\textbf{%s}\\hspace{0.5em} %s.  ``%s.'' \\textit{%s} %s%s (%s)%s. \\textsc{\\href{https://data.isiscb.org/isis/citation/%s}{%s}}.

\\vspace{2ex} \\end{footnotesize}
\\paragraph{}

"""
volume_template = " %s"
issue_template = ", %s"
pages_template = ": %s"
class Article(Document):
    def __init__(self):
        Document.__init__(self)
        self.authors = []
        self.title = ''
        self.year = 0
        self.journal = ''
        self.publisher = ''
        self.isbn = ''
        self.volume = ''
        self.issue = ''
        self.pages = ''
        
    def get_sort_by_value(self):
        return "; ".join([a.get_name_for_sort() for a in self.authors])
    
    def render(self, counter):
        authors_latex = []
        for a in self.authors:
            name = a.name
            if ',' in name:
                last = name[:name.index(',')]
                first = name[name.index(',')+1:]
            else:
                last = name[name.rfind(" "):].strip()
                first = name[:name.rfind(" ")].strip()
            authors_latex.append(author_template%(last, first))

        vol = ''
        if self.volume:
            vol = volume_template%(self.volume)
        issue = ''
        if self.issue:
            issue = issue_template%(self.issue)
        pages = ''
        if self.pages:
            pages = pages_template%(self.pages)
            
        return latex_template_article%(counter, AUTHOR_SEPARATER.join(authors_latex), self.title, self.journal, vol, issue, self.year, pages, self.isiscb_id, self.isiscb_id)


In [428]:
latex_template_chapter = """
 
\\noindent\\begin{footnotesize}\\textbf{%s}\\hspace{0.5em} %s.  ``%s.'' In \\textit{%s.}, edited by %s (%s) [ref. %s], %s. \\textsc{\\href{https://data.isiscb.org/isis/citation/%s}{%s}}.

\\vspace{2ex} \\end{footnotesize}
\\paragraph{}

"""

class Chapter(Document):
    def __init__(self):
        Document.__init__(self)
        self.authors = []
        self.editors = []
        self.title = ''
        self.year = 0
        self.publisher = ''
        self.book = None
        self.pages = ''
        
    def get_sort_by_value(self):
        return "; ".join([a.get_name_for_sort() for a in self.authors])
    
    def get_editor_string(self):
        editors_latex = []
        if self.editors:
            for e in self.editors:
                last, first = get_last_first(e.name)
                if last and first:
                    editors_latex.append(author_template%(last, first))
        eds_suffix = ""
        if editors_latex:
            eds_suffix = " (Eds.)"
            if len(editors_latex) == 1:
                eds_suffix = " (Ed.)"
        
        return AUTHOR_SEPARATER.join(editors_latex) + eds_suffix
        
        
    def render(self, counter):
        authors_latex = []
        for a in self.authors:
            name = a.name
            if ',' in name:
                last = name[:name.index(',')]
                first = name[name.index(',')+1:]
            else:
                last = name[name.rfind(" "):].strip()
                first = name[:name.rfind(" ")].strip()
            authors_latex.append(author_template%(last, first))

        return latex_template_chapter%(counter, AUTHOR_SEPARATER.join(authors_latex), self.title, self.book.title, self.book.get_editor_string() if self.book.get_editor_string() else self.get_editor_string(), self.year, self.book.index, self.pages, self.isiscb_id, self.isiscb_id)


In [429]:
latex_template_thesis = """
 
\\noindent\\begin{footnotesize}\\textbf{%s}\\hspace{0.5em} %s. ``%s.'' Dissertation at %s  (%s). \\textsc{\\href{https://data.isiscb.org/isis/citation/%s}{%s}}.

\\vspace{2ex} \\end{footnotesize}
\\paragraph{}

"""
class Thesis(Document):
    def __init__(self):
        Document.__init__(self)
        self.authors = []
        self.isbn = ''
        self.school = ''
        
    def get_sort_by_value(self):
        return "; ".join([a.get_name_for_sort() for a in self.authors])
    
    def render(self, counter):
        authors_latex = []
        for a in self.authors:
            name = a.name
            if ',' in name:
                last = name[:name.index(',')]
                first = name[name.index(',')+1:]
            else:
                last = name[name.rfind(" "):].strip()
                first = name[:name.rfind(" ")].strip()
            authors_latex.append(author_template%(last, first))

        return latex_template_thesis%(counter, AUTHOR_SEPARATER.join(authors_latex), self.title, self.school, self.year, self.isiscb_id, self.isiscb_id)

    

In [430]:
latex_template_media = """

\\noindent\\begin{footnotesize}\\textbf{%s}\\hspace{0.5em} %s. ``%s.'' (%s). \\textsc{\\href{https://data.isiscb.org/isis/citation/%s}{%s}}.

\\vspace{2ex} \\end{footnotesize}
\\paragraph{}

"""

class Media(Document):
    def __init__(self):
        Document.__init__(self)
        self.authors = []
        
    def get_sort_by_value(self):
        return "; ".join([a.get_name_for_sort() for a in self.authors])
    
    def render(self, counter):
        authors_latex = []
        for a in self.authors:
            name = a.name
            if ',' in name:
                last = name[:name.index(',')]
                first = name[name.index(',')+1:]
            else:
                last = name[name.rfind(" "):].strip()
                first = name[:name.rfind(" ")].strip()
            authors_latex.append(author_template%(last, first))

        return latex_template_media%(counter, AUTHOR_SEPARATER.join(authors_latex), self.title, self.year, self.isiscb_id, self.isiscb_id)
    

In [431]:
latex_template_essay_review = """

\\noindent\\begin{footnotesize}\\textbf{%s}\\hspace{0.5em} %s.  ``%s.'' \\textit{%s} %s%s (%s)%s. \\textsc{\\href{https://data.isiscb.org/isis/citation/%s}{%s}}.
\\begin{isisdescription}Essay review of %s, \\emph{%s}  (%s)%s.\\end{isisdescription}

\\vspace{2ex} \\end{footnotesize}
\\paragraph{}

"""

volume_template = " %s"
issue_template = ", %s"
pages_template = ": %s"
ref_template = " [ref. %s]"

class EssayReview(Document):
    def __init__(self):
        Document.__init__(self)
        self.authors = []
        self.journal = ''
        self.publisher = ''
        self.volume = ''
        self.issue = ''
        self.pages = ''
        self.book = None
     
    def get_sort_by_value(self):
        return "; ".join([a.get_name_for_sort() for a in self.authors])
   
    def render(self, counter):
        authors_latex = []
        for a in self.authors:
            name = a.name
            if ',' in name:
                last = name[:name.index(',')]
                first = name[name.index(',')+1:]
            else:
                last = name[name.rfind(" "):].strip()
                first = name[:name.rfind(" ")].strip()
            authors_latex.append(author_template%(last, first))

        vol = ''
        if self.volume:
            vol = volume_template%(self.volume)
        issue = ''
        if self.issue:
            issue = issue_template%(self.issue)
        pages = ''
        if self.pages:
            pages = pages_template%(self.pages)
            
        book_authors_latex = build_persons_latex_list(self.book.authors)
        book_editors_latex = build_persons_latex_list(self.book.editors) if type(self.book) == Book else ""
        
        book_author_editor_string = ""
        book_editors_string = ""
        if book_editors_latex:
            eds_suffix = " (Eds.)"
            if len(book_editors_latex) == 1:
                eds_suffix = " (Ed.)"
                    
        if book_authors_latex:
            book_author_editor_string = AUTHOR_SEPARATER.join(book_authors_latex)
            if book_editors_latex:
                book_editors_string = ", " + AUTHOR_SEPARATER.join(book_editors_latex) + eds_suffix
        elif book_editors_latex:
            book_author_editor_string = AUTHOR_SEPARATER.join(book_editors_latex) + eds_suffix
            
        book_ref_info = ""
        if self.book.index:
            book_ref_info = ref_template%(self.book.index)
            
        return latex_template_essay_review%(counter, AUTHOR_SEPARATER.join(authors_latex), self.title, 
                                            self.journal, vol, issue, self.year, pages, self.isiscb_id, 
                                            self.isiscb_id, book_author_editor_string, self.book.title, self.book.year, book_ref_info)

    

In [432]:
latex_template_book_review="""
\\noindent\\begin{footnotesize}\\textbf{%s}\\hspace{0.5em}%s. \\textit{%s} %s.%s

\\begin{isisdescription}%s \\end{isisdescription}

 \\end{footnotesize}

\\vspace{0.75ex}

"""

latex_book_review = "%s \\textit{%s}  %s (%s)%s."
book_review_ref_template = " [ref. %s]"

class BookReview(Document):
    def __init__(self):
        Document.__init__(self)
        self.authors = []
        self.journal = ''
        self.publisher = ''
        self.volume = ''
        self.issue = ''
        self.pages = ''
        self.reviewed_book_id = ''
        self.active = True
        
class BookReviews():
    def __init__(self):
        self.book = None
        self.reviews = []
     
    def get_sort_by_value(self):
        return "; ".join([a.get_name_for_sort() for a in self.book.authors])
   
    def get_editor_string(self):
        editors_latex = []
        if self.book.editors:
            for e in self.book.editors:
                last, first = get_last_first(e.name)
                if last and first:
                    editors_latex.append(author_template%(last, first))
        eds_suffix = ""
        if editors_latex:
            eds_suffix = " (Eds.)"
            if len(editors_latex) == 1:
                eds_suffix = " (Ed.)"
                
        return AUTHOR_SEPARATER.join(editors_latex) + eds_suffix
        
    def render(self, counter):
        authors_latex = build_persons_latex_list(self.book.authors)
        editors_latex = build_persons_latex_list(self.book.editors) if type(self.book) == Book else ""
        
        author_editor_string = ""
        editors_string = ""
        if editors_latex:
            eds_suffix = " (Eds.)"
            if len(editors_latex) == 1:
                eds_suffix = " (Ed.)"
                    
        if authors_latex:
            author_editor_string = AUTHOR_SEPARATER.join(authors_latex)
            if editors_latex:
                editors_string = ", " + AUTHOR_SEPARATER.join(editors_latex) + eds_suffix
        elif editors_latex:
            author_editor_string = AUTHOR_SEPARATER.join(editors_latex) + eds_suffix

        review_latexes = []
        for review in self.reviews:
            review_authors_latex = []
            for a in review.authors:
                name = a.name
                if ',' in name:
                    last = name[:name.index(',')]
                    first = name[name.index(',')+1:]
                else:
                    last = name[name.rfind(" "):].strip()
                    first = name[:name.rfind(" ")].strip()
                review_authors_latex.append(author_template%(last, first))
                
            
            vol = ''
            if review.volume:
                vol = volume_template%(review.volume)
        
            pages = ''
            if review.pages:
                pages = pages_template%(review.pages)
                
            review_latexes.append(latex_book_review%(AUTHOR_SEPARATER.join(review_authors_latex), review.journal, vol, review.year, pages))
            
        book_authors_latex = build_persons_latex_list(self.book.authors)
        book_editors_latex = build_persons_latex_list(self.book.editors) if type(self.book) == Book else ""
        
        book_author_editor_string = ""
        book_editors_string = ""
        if book_editors_latex:
            eds_suffix = " (Eds.)"
            if len(book_editors_latex) == 1:
                eds_suffix = " (Ed.)"
                    
        if book_authors_latex:
            book_author_editor_string = AUTHOR_SEPARATER.join(book_authors_latex)
            if book_editors_latex:
                book_editors_string = ", " + AUTHOR_SEPARATER.join(book_editors_latex) + eds_suffix
        elif book_editors_latex:
            book_author_editor_string = AUTHOR_SEPARATER.join(book_editors_latex) + eds_suffix
            
        book_ref_info = ""
        if self.book.index:
            book_ref_info = book_review_ref_template%(self.book.index)
  
        return latex_template_book_review%(counter, book_author_editor_string, self.book.title, self.book.year, book_ref_info,
                                           " ".join(review_latexes))


In [433]:
def create_author(author_string):
    author = Author()
    parts = author_string.split('||')
    for part in parts:
        part = part.strip()
        if part.startswith('AuthorityName '):
            author.name = part[len('AuthorityName '):]
            continue
        if part.startswith('ACRDisplayOrder '):
            author.order = part[len('ACRDisplayOrder '):]
    return author

In [434]:
def get_publisher(publisher_string):
    parts = publisher_string.split('||')
    for part in parts:
        part = part.strip()
        if part.startswith('AuthorityName '):
            return part[len('AuthorityName '):]

In [435]:
def build_book(row):
    book = Book()
    book.isiscb_id = row['Record ID']
    
    author_strings = row['Author'].split('//')
    for a in author_strings:
        if a.strip():
            book.authors.append(create_author(a))
        
    def get_order(author):
        return author.order
    book.authors.sort(key=get_order)
        
    editor_strings = row['Editor'].split('//')
    for e in editor_strings:
        book.editors.append(create_author(e))
    
    book.title = row['Title']
    book.year = row['Year of publication']
    book.publisher = get_publisher(row['Place Publisher'])
    book.isbn = row['ISBN']
    
    related_citations = row['Related Citations'].split('//')
    for rel_cit in related_citations:
        if rel_cit.strip():
            cit_info = rel_cit.split("||")
            cit_id = ''
            type_is_chapter = False
            for info in cit_info:
                info = info.strip()
                if info.startswith("CitationID "):
                    cit_id = info[len("CitationID "):].strip()
                if info.startswith("CCRType  "):
                    ccrType = info[len("CCRType "):].strip()
                    if ccrType.strip() == "Includes Chapter":
                        type_is_chapter = True
                    
            if type_is_chapter and cit_id:
                book.chapter_ids.append(cit_id)                    
    
    return book

In [436]:
def build_chapter(row):
    chapter = Chapter()
    chapter.isiscb_id = row['Record ID']
    
    author_strings = row['Author'].split('//')
    for a in author_strings:
        chapter.authors.append(create_author(a))
        
    def get_order(author):
        return author.order
    chapter.authors.sort(key=get_order)
    
    chapter.title = row['Title']
    chapter.year = row['Year of publication']
    
    editor_strings = row['Editor'].split('//')
    for e in editor_strings:
        chapter.editors.append(create_author(e))
    
    related_citations = row['Related Citations'].split("//")
    for cit in related_citations:
        cit_parts = cit.split("||")
        book_title = ""
        book_id = ""
        is_book_info = True
        for part in cit_parts:
            part = part.strip()
            if part.startswith("CCRType "):
                ccrType = part[len("CCRType "):]
                if ccrType.strip() != "Includes Chapter":
                    is_book_info = False
            if part.startswith("CitationTitle "):
                book_title = part[len("CitationTitle "):]
            if part.startswith("CitationID "):
                book_id = part[len("CitationID "):].strip()
                
        if is_book_info:
            chapter.book = Book()
            chapter.book.title = book_title
            chapter.book.isiscb_id = book_id
            break;
            
    pages = row['Pages Free Text']
    if pages:
        chapter.pages = pages[:pages.index("(From")].strip() if "(From" in pages else ""
                          
    return chapter        

In [437]:
def build_article(row):
    article = Article()
    
    article.isiscb_id = row['Record ID']
    
    author_strings = row['Author'].split('//')
    for a in author_strings:
        article.authors.append(create_author(a))
    
    article.title = row['Title']
    article.year = row['Year of publication']
    article.publisher = get_publisher(row['Place Publisher'])
    article.journal = row['Journal Link']
    
    vol = row['Journal Volume']
    if vol:
        article.volume = vol[:vol.index("(From")].strip() if "(From" in vol else ""
    
    issue = row['Journal Issue']
    if issue:
        article.issue = issue[:issue.index("(From")].strip() if "(From" in issue else ""
    
    pages = row['Pages Free Text']
    if pages:
        article.pages = pages[:pages.index("(From")].strip() if "(From" in pages else ""
    return article

In [438]:
def build_thesis(row):
    thesis = Thesis()
    
    thesis.isiscb_id = row['Record ID']
    
    author_strings = row['Author'].split('//')
    for a in author_strings:
        thesis.authors.append(create_author(a))
    
    thesis.title = row['Title']
    thesis.year = row['Year of publication']
    
    school_string = row['School']
    for part in school_string.split('||'):
        if part.strip().startswith('AuthorityName'):
            thesis.school = part[len("AuthorityName "):].strip()
            
    return thesis

In [439]:
def build_media(row):
    media = Media()
    media.isiscb_id = row['Record ID']
    
    author_strings = row['Author'].split('//')
    for a in author_strings:
        media.authors.append(create_author(a))
   
    media.title = row['Title']
    media.year = row['Year of publication']
    
    return media

In [440]:
def build_essay_review(row):
    essay_review = EssayReview()
    
    essay_review.isiscb_id = row['Record ID']
    
    author_strings = row['Author'].split('//')
    for a in author_strings:
        essay_review.authors.append(create_author(a))
    
    essay_review.title = row['Title']
    essay_review.year = row['Year of publication']
    essay_review.publisher = get_publisher(row['Place Publisher'])
    essay_review.journal = row['Journal Link']
    
    vol = row['Journal Volume']
    if vol:
        essay_review.volume = vol[:vol.index("(From")].strip() if "(From" in vol else ""
    
    issue = row['Journal Issue']
    if issue:
        essay_review.issue = issue[:issue.index("(From")].strip() if "(From" in issue else ""
    
    pages = row['Pages Free Text']
    if pages:
        essay_review.pages = pages[:pages.index("(From")].strip() if "(From" in pages else ""
        
    related_citations = row['Related Citations'].split("//")
    for cit in related_citations:
        cit_parts = cit.split("||")
        book_title = ""
        book_id = ""
        is_book_info = True
        for part in cit_parts:
            part = part.strip()
            if part.startswith("CCRType "):
                ccrType = part[len("CCRType "):]
                if ccrType.strip() != "Is Reviewed By":
                    is_book_info = False
            if part.startswith("CitationTitle "):
                book_title = part[len("CitationTitle "):]
            if part.startswith("CitationID "):
                book_id = part[len("CitationID "):].strip()
                
        if is_book_info:
            essay_review.book = Book()
            essay_review.book.title = book_title
            essay_review.book.isiscb_id = book_id
            break;
            
    return essay_review

In [441]:
def build_book_review(row):
    book_review = BookReview()
    
    book_review.isiscb_id = row['Record ID']
    
    author_strings = row['Author'].split('//')
    for a in author_strings:
        if a.strip():
            book_review.authors.append(create_author(a))
        
    def get_order(author):
        return author.order
    book_review.authors.sort(key=get_order)
    
    book_review.year = row['Year of publication']
    book_review.journal = row['Journal Link']
    
    vol = row['Journal Volume']
    if vol:
        book_review.volume = vol[:vol.index("(From")].strip() if "(From" in vol else ""
    
    issue = row['Journal Issue']
    if issue:
        book_review.issue = issue[:issue.index("(From")].strip() if "(From" in issue else ""
    
    pages = row['Pages Free Text']
    if pages:
        book_review.pages = pages[:pages.index("(From")].strip() if "(From" in pages else ""
        
    status = row['Record Nature']
    if not status.startswith('Active'):
        book_review.active = False
        
    related_citations = row['Related Citations'].split("//")
    for cit in related_citations:
        cit_parts = cit.split("||")
        book_title = ""
        book_id = ""
        is_book_info = True
        for part in cit_parts:
            part = part.strip()
            if part.startswith("CCRType "):
                ccrType = part[len("CCRType "):]
                if ccrType.strip() != "Is Reviewed By":
                    is_book_info = False
            if part.startswith("CitationTitle "):
                book_title = part[len("CitationTitle "):]
            if part.startswith("CitationID "):
                book_id = part[len("CitationID "):].strip()
                
        if is_book_info:
            book_review.reviewed_book_id = book_id
            break;
            
    return book_review

In [442]:
class Classification():
    def __init__(self):
        self.code = ''
        self.code_print = ''
        self.code_for_sorting_one = -1
        self.code_for_sorting_two = -1
        self.name = ''

In [443]:
with open(export_file) as csvfile:
    reader = csv.DictReader(csvfile)
    docs = []
    docs_by_id = {}
    reviews_by_book_id = {}
    
    for row in reader:
        doc = None
        if row['Record Type'] == "Book":
            doc = build_book(row)
        elif row['Record Type'] == "Article":
            doc = build_article(row)
        elif row['Record Type'] == "Chapter":
            doc = build_chapter(row)
        elif row['Record Type'] == "Thesis":
            doc = build_thesis(row)
        elif row['Record Type'] == "Multimedia Object":
            doc = build_media(row)
        elif row['Record Type'] == "Essay Review":
            doc = build_essay_review(row)
        elif row['Record Type'] == "Review":
            doc = build_book_review(row)

        if doc and not doc.isiscb_id in docs_by_id:
            doc.category_number = row['CategoryNumbers'].strip()
            doc.print_date = row['Published Print'].strip()
            if type(doc) != BookReview:
                docs.append(doc)
                docs_by_id[doc.isiscb_id] = doc
                
                # classification
                classi = Classification()
                class_codes = row['CategoryNumbers'].split('//')
                doc.classifications = []
                for class_code in class_codes:
                    for part in class_code.split('||'):
                        if part.strip().startswith('ClassificationCode'):
                            code_parts = part[len("ClassificationCode "):].strip().split('-')
                            classi.code = part[len("ClassificationCode "):].strip()
                            while "" in code_parts: code_parts.remove("")
                            if len(code_parts) == 1:
                                classi.code_for_sorting_one = int(code_parts[0].strip())
                                classi.code_print = code_parts[0].strip()
                            if len(code_parts) == 2:
                                classi.code_for_sorting_one = int(code_parts[1].strip())
                                classi.code_for_sorting_two = int(code_parts[0].strip())
                                classi.code_print = code_parts[1].strip() + "-" + code_parts[0].strip()
                    doc.classifications.append(classi)
            else:
                reviews_list = reviews_by_book_id.get(doc.reviewed_book_id, None)
                if not reviews_list:
                    reviews_by_book_id[doc.reviewed_book_id] = []
                
                if doc.active and not doc.print_date:
                    reviews_by_book_id[doc.reviewed_book_id].append(doc)
            
    def get_sort_by_value(doc):
        return doc.get_sort_by_value()

    docs.sort(key=get_sort_by_value)
    counter = 1
    for doc in docs:
        if doc.category_number and not doc.print_date:
            doc.index = counter
            counter += 1
        
    with open(result_file_path, 'w') as result_file:
        for doc in docs:
            if (type(doc) in [Chapter, EssayReview]) and doc.book.isiscb_id in docs_by_id:
                book = docs_by_id[doc.book.isiscb_id]
                if book:
                    doc.book = book
            if type(doc) == Book and doc.chapter_ids:
                for chap_id in doc.chapter_ids:
                    if chap_id in docs_by_id:
                        doc.chapters.append(docs_by_id[chap_id])
            # print all classified records (e.g., have Category Number) 
            # that have not been previously printed (e.g., have Published Print data).
            if doc.category_number and not doc.print_date:
                print("Writing " + str(doc.isiscb_id))
                for c in doc.classifications:
                    print(c.code_print)
                result_file.write(doc.render(doc.index).strip() + "\n")
            
    all_book_reviews = []
    for book_id in reviews_by_book_id:
        if book_id in docs_by_id:
            book = docs_by_id[book_id]

            if book:
                book_reviews = BookReviews()
                book_reviews.book = book
                book_reviews.reviews = reviews_by_book_id[book_id]
                all_book_reviews.append(book_reviews)
            
    def get_sort_by_value(doc):
        return doc.get_sort_by_value()

    all_book_reviews.sort(key=get_sort_by_value)
    counter = 1
    for doc in all_book_reviews:
        doc.index = "R" + str(counter)
        counter += 1
            
    with open(reviews_file_path, 'w') as reviews_file:
        for reviews in all_book_reviews:
            print("Writing Reviews for" + str(reviews.book.isiscb_id))
            reviews_file.write(reviews.render(reviews.index).strip() + "\n")


Writing CBB252593054
370-137
Writing CBB817048179
230-103
Writing CBB387203093
360-160
Writing CBB659070972
2
Writing CBB466150778
380-5
Writing CBB195418610
380-146
Writing CBB539791194
380-146
Writing CBB900733613
163
Writing CBB743149463
200-110
Writing CBB991473793
230-20
Writing CBB232156804
370-160
370-160
Writing CBB023904641
6
Writing CBB634939172
42
Writing CBB260495673
360-113
Writing CBB664016347
360-23
Writing CBB007618719
312-110
Writing CBB066579010
380-43
Writing CBB502965218
42
Writing CBB087426117
380-42
Writing CBB702535308
330-23
Writing CBB414752960
370-164
Writing CBB903684763
375-161
Writing CBB313774069
375-152
Writing CBB794239572
380-42
Writing CBB028067262
210-103
Writing CBB346881103
360-103
Writing CBB644056501
370-150
Writing CBB348764862
360-103
Writing CBB812551579
350-12
Writing CBB979681231
380-150
Writing CBB654318221
330-102
Writing CBB762575960
350-41
Writing CBB114433718
230-22
Writing CBB416193584
380-133
Writing CBB185165933
330-152
Writing CBB678

Writing CBB491994379
370-29
Writing CBB451296542
340-161
Writing CBB788081102
370-160
Writing CBB513731274
370-160
Writing CBB217696360
330-23
Writing CBB431012136
160
Writing CBB750960782
360-160
Writing CBB191691606
375-161
Writing CBB924063755
360-110
Writing CBB008372474
370-160
Writing CBB741842597
370-123
Writing CBB128236614
230-150
Writing CBB519729121
375-22
Writing CBB527216150
330-121
Writing CBB374532018
350-102
Writing CBB193379455
375-153
Writing CBB947819241
360-23
Writing CBB532696700
360-1
Writing CBB114385437
375-22
Writing CBB620089033
330-1
Writing CBB572326722
260-110
Writing CBB839031957
360-103
Writing CBB886350845
350-23
Writing CBB168492556
27
Writing CBB137166866
312-137
Writing CBB163647651
340-110
Writing CBB805816624
320-114
Writing CBB727175869
375-160
Writing CBB712303405
370-163
Writing CBB621114449
320-111
Writing CBB915420021
375-29
Writing CBB232039985
370-42
Writing CBB030932589
370-22
Writing CBB078148558
360-10
Writing CBB962411591
121
Writing CBB7

Writing CBB505969922
1
Writing CBB306072813
1
Writing CBB579357935
200-22
Writing CBB467951276
312-110
Writing CBB088100681
375-134
Writing CBB735543971
1
Writing CBB480712244
42
Writing CBB056885812
370-160
Writing CBB394194579
380-161
Writing CBB892284362
160
Writing CBB101471819
370-10
Writing CBB542876607
312-28
312-28
Writing CBB494220355
360-121
Writing CBB930351745
370-164
Writing CBB933911142
370-120
Writing CBB218621435
380-10
Writing CBB581222956
42
Writing CBB721793137
360-163
Writing CBB298943430
370-150
Writing CBB634311326
370-160
Writing CBB660017555
350-160
Writing CBB329031708
370-161
Writing CBB826755445
370-135
Writing CBB761069376
380-160
Writing CBB658584104
160
Writing CBB514982523
375-140
Writing CBB464920868
360-10
Writing CBB122831509
311-150
Writing CBB324730623
370-161
Writing CBB037378609
330-121
Writing CBB459434748
380-23
Writing CBB631225601
370-160
Writing CBB888869966
10
Writing CBB093085222
375-152
Writing CBB460567152
160
Writing CBB842859312
380-6
Wr

Writing CBB567238478
370-121
Writing CBB785502954
370-121
Writing CBB222382824
370-121
Writing CBB039612869
370-121
Writing CBB892348320
370-121
Writing CBB603774278
360-121
Writing CBB947608759
360-28
Writing CBB887119766
370-160
Writing CBB265828740
370-160
Writing CBB327523399
360-150
Writing CBB460220882
260-145
Writing CBB160571215
111
Writing CBB726347258
360-23
Writing CBB784211091
360-160
Writing CBB550400827
350-160
Writing CBB696844357
42
Writing CBB975752669
360-121
Writing CBB479920463
370-150
Writing CBB970702810
370-29
Writing CBB266260240
360-150
Writing CBB210350336
350-150
Writing CBB863073003
160
Writing CBB740750197
380-160
380-160
Writing CBB467220873
123
Writing CBB558341798
121
Writing CBB019688623
370-130
Writing CBB038559257
360-121
Writing CBB772513552
230-150
Writing CBB514429550
330-121
Writing CBB315812764
370-121
Writing CBB445923301
360-113
Writing CBB311723811
380-23
Writing CBB415715597
300-110
Writing CBB836448854
380-42
Writing CBB836143752
330-121
Wri